<a href="https://colab.research.google.com/github/bdesgarennes/NFLAnalytics-/blob/main/NFL_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NFL Player Analytics 
### Braden desGarennes & Maksim Kalmanovich
### February 2023

**Reasoning:** A very common held "fact" seen around Football and Twitter is that the QB is a football team's most valuable position and is the position that ulitmitly leads to the most wins. This can be seen, not only if you ask a fan, coach, or player but is clearly represented in the NFL's MVPs going to QBs in 46/67 times and Super Bowl MVPS going to QBs 32/57 times. We wanted to test this theory and see if the data actually backs this up or if this common belief is in fact innacurate.   

**Hypothesis:** Quarterbacks have the most impact on wins on the football field.

[NFL News: Ranking each position's importance, from quarterback to returner](https://www.nfl.com/news/ranking-each-position-s-importance-from-quarterback-to-returner-0ap3000000503855)

[Football Positions Ranked by Importance](https://www.rookieroad.com/football/positions/positions-ranked-by-importance/#:~:text=The%20QB%20is%20the%20most,make%20decisions%20every%20single%20play.)

[Bleacher Report: Power Ranking the Importance of Each Position in Todays NFL](https://bleacherreport.com/articles/1659834-power-ranking-the-importance-of-each-position-in-todays-nfl)

In [ ]:
# first some initial setup of libraries we will be using
import pandas as pd # this is the main library used in python for manipulating data. 
import json # this is a library to parse json (javascript object notation)
from urllib.request import urlopen # library to read data from a URL
import matplotlib.pyplot as plt # visualization library we will use that integrates with pandas

In [ ]:
url="https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv"
elo_df = pd.read_csv(url)
elo_df.head

url="https://projects.fivethirtyeight.com/nfl-api/nfl_elo_latest.csv"
elo_latest_df = pd.read_csv(url)
elo_latest_df.head

<bound method NDFrame.head of            date  season  neutral playoff team1 team2     elo1_pre  \
0    2022-09-08    2022        0     NaN   LAR   BUF  1614.761436   
1    2022-09-11    2022        0     NaN   MIA    NE  1539.979483   
2    2022-09-11    2022        0     NaN   CIN   PIT  1558.193285   
3    2022-09-11    2022        0     NaN   ATL    NO  1436.671709   
4    2022-09-11    2022        0     NaN   NYJ   BAL  1364.983381   
..          ...     ...      ...     ...   ...   ...          ...   
279  2023-01-22    2022        0       d   BUF   CIN  1731.516093   
280  2023-01-22    2022        0       d    SF   DAL  1708.971847   
281  2023-01-29    2022        0       c   PHI    SF  1621.987292   
282  2023-01-29    2022        0       c    KC   CIN  1735.804844   
283  2023-02-12    2022        1       s   PHI    KC  1657.913193   

        elo2_pre  elo_prob1  elo_prob2  ...  qb2_game_value  qb1_value_post  \
0    1614.246762   0.593181   0.406819  ...      402.149420   